In [1]:
from tensorflow.keras.datasets import imdb
import numpy as np
import tensorflow as tf

In [2]:
NUM_WORDS = 10000
MAX_LENGHT = 25

In [3]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS)

# Inspect data

Train data is an array of integer indexes

Label 1 for possitive and 0 for negative feelings towards the review

In [4]:
train_data.shape

(25000,)

In [5]:
train_data[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [6]:
train_labels[0]

1

# translate sentence to words

In [7]:
index_to_word = dict([(value, key) for key, value in imdb.get_word_index().items()])

In [8]:
index_to_word[1]

'the'

In [9]:
" ".join([index_to_word[index] for index in train_data[0]])

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have t

# 1st Approach using 1 hot encoding

In [10]:
type(train_data)

numpy.ndarray

In [11]:
for s in train_data:
    print(type(s))
    break

<class 'list'>


In [12]:
def padding(sentence, max_lenght):
    if max_lenght:
        return sentence[:max_lenght]
    return sentence

def onehot_v2(sentences, num_words, max_lenght):
    """
        For each sentence I have a word index
        4 7 43 314
        then limit those words to max lenght
        the last step is to place a 1 in the position of each word Index
        the output will be a 10000 position array
        the array goes from 0 so position 4 will be the 5th position of the array
        0 0 0 1 0 0 1 0 0 0 0 0 .... (44)1 ... (315)1
    """
    zeros_array = np.zeros((len(sentences), num_words))
    for sentence_index, sentence in enumerate(sentences):
        for word_index in padding(sentence, max_lenght=max_lenght):
            zeros_array[sentence_index, word_index] = 1
    
    return zeros_array

#padding was not working
def onehot_v1(sentences, num_words, max_lenght):
    """
        For each sentence I have a word index
        4 7 43 314
        then limit those words to max lenght
        the last step is to place a 1 in the position of each word Index
        the output will be a 10000 position array
        the array goes from 0 so position 4 will be the 5th position of the array
        0 0 0 1 0 0 1 0 0 0 0 0 .... (44)1 ... (315)1
    """
    zeros_array = np.zeros((len(sentences), num_words))
    for sentence_index, sentence in enumerate(padding(sentences, max_lenght=max_lenght)):
        for word_index in sentence:
            zeros_array[sentence_index, word_index] = 1
    
    return zeros_array

def onehot_v2(sentences, num_words, max_lenght):
    """
        For each sentence I have a word index
        4 7 43 314
        then limit those words to max lenght
        the last step is to place a 1 in the position of each word Index
        the output will be a 10000 position array
        the array goes from 0 so position 4 will be the 5th position of the array
        0 0 0 1 0 0 1 0 0 0 0 0 .... (44)1 ... (315)1
    """
    zeros_array = np.zeros((len(sentences), num_words))
    for sentence_index, sentence in enumerate(sentences):
        for word_index in padding(sentence, max_lenght=max_lenght):
            zeros_array[sentence_index, word_index] = 1
    
    return zeros_array

In [13]:
one_hot_train_data = onehot_v2(train_data, max_lenght=MAX_LENGHT, num_words=NUM_WORDS)

In [14]:
one_hot_train_data.shape

(25000, 10000)

# 0 is reserved for the UKN token

In [15]:
one_hot_test_data = onehot_v2(test_data, max_lenght=MAX_LENGHT, num_words=NUM_WORDS)

In [16]:
one_hot_test_data.shape

(25000, 10000)

# one hot labels

In [17]:
one_hot_test_label = np.array(test_labels, dtype="float32")
one_hot_train_label = np.array(train_labels, dtype="float32")

# Test on hot encoding

In [18]:
MAX_LENGHT

25

In [19]:
type(train_data)

numpy.ndarray

In [20]:
type(train_data[1])

list

In [21]:
padding(train_data[1], 5)

[1, 194, 1153, 194, 8255]

In [22]:
train_data[1][:5]

[1, 194, 1153, 194, 8255]

# Bug padding was not working

In [23]:
onehot_v1([train_data[1]], NUM_WORDS, 5)

array([[0., 1., 1., ..., 0., 0., 0.]])

In [24]:
onehot_v2([train_data[1]], NUM_WORDS, 5)

array([[0., 1., 0., ..., 0., 0., 0.]])

# Model

In [25]:
X_inputs = tf.keras.Input(shape=(NUM_WORDS))
X = tf.keras.layers.Dense(units=32, activation="relu")(X_inputs)
X = tf.keras.layers.Dense(units=16, activation="relu")(X_inputs)
X = tf.keras.layers.Dense(units=8, activation="relu")(X_inputs)
X = tf.keras.layers.Dense(units=4, activation="relu")(X_inputs)
X_output = tf.keras.layers.Dense(units=1, activation="sigmoid")(X)

model=tf.keras.Model(inputs=X_inputs, outputs=X_output)

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



2023-01-05 11:41:42.442971: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-05 11:41:42.443119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [26]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.RMSprop(lr=1e-3),
                metrics=["accuracy", "Precision", "Recall", "AUC"])

/Users/matiasgonzalez/miniforge3/envs/tf_2_9_metal_0_5_keras/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


# Callbacks

In [27]:
early_stop = tf.keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience = 5
            )

def scheduler(epoch, lr):
  if epoch < 15:
    return lr
  else:
    return lr * tf.math.exp(-0.05)

learning_rate_exp_reduce = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(save_best_only=True, filepath="checkpoint")

In [28]:
model.fit(one_hot_train_data,
         one_hot_train_label,
         epochs=500,
         validation_data=(one_hot_test_data, one_hot_test_label),
         callbacks=[early_stop, learning_rate_exp_reduce, model_checkpoint])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/500


2023-01-05 11:41:43.126071: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-05 11:41:43.428599: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - ETA: 0s - loss: 0.6066 - accuracy: 0.6785 - precision: 0.6792 - recall: 0.6766 - auc: 0.7489

2023-01-05 11:41:56.269310: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: checkpoint/assets
782/782 [==============================] - 22s 27ms/step - loss: 0.6066 - accuracy: 0.6785 - precision: 0.6792 - recall: 0.6766 - auc: 0.7489 - val_loss: 0.5620 - val_accuracy: 0.7072 - val_precision: 0.7101 - val_recall: 0.7001 - val_auc: 0.7852 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/500
782/782 [==============================] - 18s 24ms/step - loss: 0.5115 - accuracy: 0.7513 - precision: 0.7439 - recall: 0.7665 - auc: 0.8300 - val_loss: 0.5413 - val_accuracy: 0.7188 - val_precision: 0.7206 - val_recall: 0.7145 - val_auc: 0.7985 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/500
782/782 [==============================] - 19s 24ms/step - loss: 0.4763 - accuracy: 0.7688 - precision: 0.7603 - recall: 0.7850 - auc: 0.8531 - val_loss: 0.5497 - val_accuracy: 0.7170 - val_precision: 0.7401 - val_recall: 0.6690 - val_auc: 0.8